#### Automated Speeding Infringement Notice

----

In real world scenarios where companies are providing managed fleets, sometimes the company collects metrics for safety and insurance - such as speed monitoring

### instructions

Run each cell of code in order to test the messaging system and make the rover LEDs blink!

In [ ]:
# install python libraries
%pip install -r ../requirements.txt

In [ ]:
# import python libraries
import smtplib
import ssl
import paho.mqtt.client as mqtt
import time
from python.infringement_helper import InfringementHelper

In [ ]:
# DECLARE GLOBALS
# messaging system connection details
BROKER_HOSTNAME = "localhost"
BROKER_HOSTNAME = "robotics14.lan.robolab"
BROKER_HOSTNAME = "192.168.1.114"

# messaging topics
SPEED_TOPIC = "spheroRVR/speed"
LED_BLINK_TOPIC = "spheroRVR/led/blink"
LED_GREEN_TOPIC = "spheroRVR/led/green"

# The speed limit
MAXIMUM_SPEED_LIMIT = 0.4

In [ ]:
# set up minimal messaging service functions
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to broker")

        global Connected  # Use global variable
        Connected = True  # Signal connection

    else:
        print(rc)
        print("Connection failed")


In [ ]:
# the email function
def send_email():
    # Set up your Gmail account details
    sender_email = 'engrehsanamiri@gmail.com'
    receiver_email = 'engrehsanamiri@gmail.com'
    password = 'jlaezogdlhjnllwu'
    subject = 'Test email'
    body = 'This is a test email sent from Python.'
    msg = f'Subject: {subject}\n\n{body}'

    # Set up the email message
    message = """\
    Subject: Test email from Python

    This is a test email sent from Python."""

    # Create a secure SSL context
    context = ssl.create_default_context()

    try:
        # Log in to your Gmail account and create a secure connection
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
            server.login(sender_email, password)

            # Send the email message
            server.sendmail(sender_email, receiver_email, msg)

        print('Email sent successfully.')
    except Exception as e:
        print('Error sending email: %s', e)

In [ ]:
# create a tracking object to help track infringement events
infringement_tracker = InfringementHelper(MAXIMUM_SPEED_LIMIT)

In [ ]:
# this function helps us listen to messages from the rover
# when we get a message about speed, we can check if it is too high
def on_message(client, userdata, message):
    print("speed message received! -> " + str(float(message.payload)))

    speed = float(message.payload)
    
    # check and track speeding events
    infringement_tracker.track_infringement(speed)
    
    if infringement_tracker.speeding_event_finished():
        notice_text = "hello"
        infringement_tracker.send_infringement_notice(notice_text)

#### The Main Program

---

This function is the *main* function. It connects to the messaging service and set's up an automated response to messages from the rover. If the rover sends it's speed and it is too high then an email will be send for speeding

In [ ]:
def main():
    """ This program listens to rover messages and notifies of traffic infringements using mqtt and email
    """

    # Set up the MQTT client
    client = mqtt.Client("remote")
    client.on_connect = on_connect
    client.on_message = on_message

    # Connect to the MQTT broker
    client.connect(BROKER_HOSTNAME, 1883)
    client.loop_start()  # start the loop

    # subscribe to a message on a topic
    client.subscribe(SPEED_TOPIC)

    try:
        while True:
            print("listening...")
            time.sleep(1)

    except KeyboardInterrupt:
        print("exiting")
        client.disconnect()
        client.loop_stop()


In [ ]:
# test the main program
main()